Estructura que vamos a implementar

Generador de posiciones de vehiculos
Revisar la función Treading.Thread, genera la misma función de python varias veces a la vez

In [ ]:
# Aqui deberíamos poner el generador que tenemos, como idea, podemos guardar los mensajes en un bucket o mandarlos a pupsug, 
# revisar la función que genera varios posiciones random de diferentes vehículos como en el postwork de Javi.
# En el trabajo final, esto lo puede hacer un VM

Generador de posiciones de individuos
Revisar la función Treading.Thread, genera la misma función de python varias veces a la vez

In [ ]:
# Hacer otro generador, que tomando las posiciones de los kml´s y añadiendo  +/- x puntos, nos de posiciones 
# random de los usuarios. Podemos tambien guardarlos en otro bucket para posteriormente leerlos de ahí o mandarlos a pupsug.
# En el trabajo final, esto lo puede hacer un VM 

Colas de mensajes. Tendremos así dos colas de PupSub que leerán los mensajes de los dos buckets: vehículos y clientes

In [ ]:
        """ Part 01: Read data from PubSub. """

        data1 = (
            p
                | "Read From PubSub" >> # leemos del bucket de vehiculos
                | "Parse JSON messages" >> #Tenemos que ver cual es la estructura del mensaje y transformarlo a nuetra conveniencia
        )

        data2 = (
            p
                | "Read From PubSub" >> # leemos del bucket de clientes
                | "Parse JSON messages" >> #Tenemos que ver cual es la estructura del mensaje y transformarlo a nuetra conveniencia
        )

        """ Part 02: Get the aggregated data of the vehicle within the section. """

        data1, data2, processed_data = (
            
             
                | "tenemos que leer los datos de ambos topics" >> #ToDo: Complete this section
                | "Cramos una ventana temporal donde" >> # Tendremos las posiciones de los vehiculos y de los clientes
                | "Creamos una funcion (Dofn)" >> # Donde comparemos las distancias entre ambos
                # si esta distancia es > que x match, sino, no match.
                | "Tendremos que guardar los match en bigquery" >> 
                |"Ver donde guardamos los no match si en bigquery o en otro bucket sin más"
        
    

Leeremos de bigquery los resultados match y los mostramos con streamlit.

en local

In [15]:
from flask import Flask, request, jsonify
import os
import random
import json
import time
import google.cloud
from google.cloud import pubsub_v1
from google.cloud import storage
from google.cloud.storage import Blob
import xml.etree.ElementTree as ET
import argparse
import logging

In [12]:

folder_path = 'C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas'
files = [('files[]', (file, open(os.path.join(folder_path, file), 'rb'))) for file in os.listdir(folder_path) if file.endswith('.kml')]
print(files)


[('files[]', ('1.kml', <_io.BufferedReader name='C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas\\1.kml'>)), ('files[]', ('10.kml', <_io.BufferedReader name='C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas\\10.kml'>)), ('files[]', ('11.kml', <_io.BufferedReader name='C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas\\11.kml'>)), ('files[]', ('12.kml', <_io.BufferedReader name='C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas\\12.kml'>)), ('files[]', ('13.kml', <_io.BufferedReader name='C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas\\13.kml'>)), ('files[]', ('14.kml', <_io.BufferedReader name='C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas\\14.kml'>)), ('files[]', ('15.kml', <_io.BufferedReader name='C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas\\15.kml'>)), ('files[]', ('16.kml', <_io.BufferedReader name='C:/Users/josan/Documents/Git

In [22]:
def get_coords_finalesVehiculosClientes(bucket_name, DOWNLOAD_FOLDER):
    latitudes_finales = []
    longitudes_finales = []

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    blobs = bucket.list_blobs()

    for blob in blobs:
        if blob.name.endswith(".kml"):
            kml_file = os.path.join(DOWNLOAD_FOLDER, blob.name)

            with open(kml_file, "wb") as file:
                blob.download_to_file(file)

            with open(kml_file, "r", encoding="utf-8") as archivo:
                datos_kml = archivo.read()

            raiz = ET.fromstring(datos_kml)
            coordenadas = raiz.findall(".//{http://www.opengis.net/kml/2.2}coordinates")

            if coordenadas:
                for coordenada_str in coordenadas:
                    coords_lista = [tuple(map(float, _.split(','))) for _ in coordenada_str.text.split()]
                    for coords in coords_lista:
                        longitudes_finales.append(coords[0])
                        latitudes_finales.append(coords[1])

    return latitudes_finales, longitudes_finales

In [46]:
import os
import xml.etree.ElementTree as ET

def get_coords_finalesVehiculos(local_folder):
    datos_ofertas = []

    for file_name in os.listdir(local_folder):
        if file_name.endswith(".kml"):
            kml_file = os.path.join(local_folder, file_name)

            with open(kml_file, "r", encoding="utf-8") as file:
                datos_kml = file.read()

            raiz = ET.fromstring(datos_kml)
            coordenadas = raiz.findall(".//{http://www.opengis.net/kml/2.2}coordinates")

            if coordenadas:
                for coordenada_str in coordenadas:
                    coords_list = [tuple(map(float, _.split(','))) for _ in coordenada_str.text.split()]

                    for i, coords in enumerate(coords_list):
                        data1 = {}
                        data1["id_oferta"] = file_name  # Podrías cambiar esto si tienes un identificador de oferta diferente
                        data1["punto"] = i + 1
                        data1["latitude"] = coords[1]
                        data1["longitude"] = coords[0]
                        datos_ofertas.append(data1)

    return datos_ofertas

# Llamada a la función con el directorio local
datos_demandas = get_coords_finalesVehiculos("C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas")

# Imprimir o hacer algo con los datos_ofertas
for data1 in datos_demandas:
    # time.sleep(1)
    print(data1)


{'id_oferta': '1.kml', 'punto': 1, 'latitude': 39.46126, 'longitude': -0.34924}
{'id_oferta': '1.kml', 'punto': 2, 'latitude': 39.46136, 'longitude': -0.34954}
{'id_oferta': '1.kml', 'punto': 3, 'latitude': 39.46154, 'longitude': -0.35008}
{'id_oferta': '1.kml', 'punto': 4, 'latitude': 39.46185, 'longitude': -0.35114}
{'id_oferta': '1.kml', 'punto': 5, 'latitude': 39.46197, 'longitude': -0.35152}
{'id_oferta': '1.kml', 'punto': 6, 'latitude': 39.46227, 'longitude': -0.35246}
{'id_oferta': '1.kml', 'punto': 7, 'latitude': 39.46227, 'longitude': -0.35247}
{'id_oferta': '1.kml', 'punto': 8, 'latitude': 39.46232, 'longitude': -0.35266}
{'id_oferta': '1.kml', 'punto': 9, 'latitude': 39.46225, 'longitude': -0.35269}
{'id_oferta': '1.kml', 'punto': 10, 'latitude': 39.46129, 'longitude': -0.35309}
{'id_oferta': '1.kml', 'punto': 11, 'latitude': 39.46065, 'longitude': -0.35333}
{'id_oferta': '1.kml', 'punto': 12, 'latitude': 39.46065, 'longitude': -0.35335}
{'id_oferta': '1.kml', 'punto': 13, '

In [45]:
import os
import xml.etree.ElementTree as ET

def get_coords_finalesClientes(local_folder):
    datos_demandas = []

    for file_name in os.listdir(local_folder):
        if file_name.endswith(".kml"):
            kml_file = os.path.join(local_folder, file_name)

            with open(kml_file, "r", encoding="utf-8") as file:
                datos_kml = file.read()

            raiz = ET.fromstring(datos_kml)
            coordenadas = raiz.findall(".//{http://www.opengis.net/kml/2.2}coordinates")

            if coordenadas:
                for coordenada_str in coordenadas:
                    coords_list = [tuple(map(float, _.split(','))) for _ in coordenada_str.text.split()]

                    for i, coords in enumerate(coords_list):
                        data2 = {}
                        data2["id_demanda"] = file_name  # Podrías cambiar esto si tienes un identificador de oferta diferente
                        data2["punto"] = i + 1
                        data2["latitude"] = coords[1]
                        data2["longitude"] = coords[0]
                        datos_demandas.append(data2)

    return datos_demandas

# Llamada a la función con el directorio local
datos_ofertas = get_coords_finalesClientes("C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas")

# Imprimir o hacer algo con los datos_ofertas
for data2 in datos_demandas:
    # time.sleep(1)
    print(data2)

{'id_demanda': '1.kml', 'punto': 1, 'latitude': 39.46126, 'longitude': -0.34924}
{'id_demanda': '1.kml', 'punto': 2, 'latitude': 39.46136, 'longitude': -0.34954}
{'id_demanda': '1.kml', 'punto': 3, 'latitude': 39.46154, 'longitude': -0.35008}
{'id_demanda': '1.kml', 'punto': 4, 'latitude': 39.46185, 'longitude': -0.35114}
{'id_demanda': '1.kml', 'punto': 5, 'latitude': 39.46197, 'longitude': -0.35152}
{'id_demanda': '1.kml', 'punto': 6, 'latitude': 39.46227, 'longitude': -0.35246}
{'id_demanda': '1.kml', 'punto': 7, 'latitude': 39.46227, 'longitude': -0.35247}
{'id_demanda': '1.kml', 'punto': 8, 'latitude': 39.46232, 'longitude': -0.35266}
{'id_demanda': '1.kml', 'punto': 9, 'latitude': 39.46225, 'longitude': -0.35269}
{'id_demanda': '1.kml', 'punto': 10, 'latitude': 39.46129, 'longitude': -0.35309}
{'id_demanda': '1.kml', 'punto': 11, 'latitude': 39.46065, 'longitude': -0.35333}
{'id_demanda': '1.kml', 'punto': 12, 'latitude': 39.46065, 'longitude': -0.35335}
{'id_demanda': '1.kml', '

Creamos un pipeline de vehiculos:

In [28]:
!pip show apache_beam

Name: apache-beam
Version: 2.51.0
Summary: Apache Beam SDK for Python
Home-page: https://beam.apache.org
Author: Apache Software Foundation
Author-email: dev@beam.apache.org
License: Apache License, Version 2.0
Location: c:\users\josan\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages
Requires: cloudpickle, crcmod, dill, fastavro, fasteners, grpcio, hdfs, httplib2, js2py, numpy, objsize, orjson, packaging, proto-plus, protobuf, pyarrow, pydot, pymongo, python-dateutil, pytz, regex, requests, typing-extensions, zstandard
Required-by: 


In [29]:
# Import Python Libraries
import logging
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

In [67]:
from datetime import datetime

datos_demandas = get_coords_finalesClientes("C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas")
datos_ofertas = get_coords_finalesVehiculos("C:/Users/josan/Documents/GitHub/DATA_PROJECT_2/Ejecutable/coordenadas")

class TransformarMensaje(beam.DoFn):
    def process(self, element):
        # Extraer la latitud y longitud del elemento
        latitude = element['latitude']
        longitude = element['longitude']

        # Crear la tupla de dos elementos
        tupla_lat_long = (latitude, longitude)

        # Devolver el mensaje transformado
        return [(tupla_lat_long, element)]



# Crear un pipeline de Apache Beam
with beam.Pipeline(InteractiveRunner()) as p:
    # Leer los datos_ofertas usando la transformación Create
    ofertas_vehiculos= ( 
        p   
        | "Create PCollection" >> beam.Create(datos_ofertas)# datos
        | "Transformamos los datos con el Dofn">> beam.ParDo(TransformarMensaje())
        # | "Agrupamos por clave" >> beam.GroupByKey()
        
    )
    demandas_vehiculos= ( 
        p   
        | "Create PCollection2" >> beam.Create(datos_demandas)# datos
        | "Transformamos los datos con el Dofn2" >> beam.ParDo(TransformarMensaje())
        # | "Agrupamos por clave2" >> beam.GroupByKey()
        
    )
    combined_pcollection = (
        (ofertas_vehiculos, demandas_vehiculos)
        | "Creamos una tercera Pcollection a partir de las anteriores">> beam.CoGroupByKey()
        | "Agrupamos por key" >> beam.GroupByKey() 
        | "Imprimimos" >>beam.Map(print) 
        )


   
   

        





    

((39.46126, -0.34924), [([{'id_oferta': '1.kml', 'punto': 1, 'latitude': 39.46126, 'longitude': -0.34924}, {'id_oferta': '12.kml', 'punto': 136, 'latitude': 39.46126, 'longitude': -0.34924}, {'id_oferta': '5.kml', 'punto': 109, 'latitude': 39.46126, 'longitude': -0.34924}], [{'id_demanda': '1.kml', 'punto': 1, 'latitude': 39.46126, 'longitude': -0.34924}, {'id_demanda': '12.kml', 'punto': 136, 'latitude': 39.46126, 'longitude': -0.34924}, {'id_demanda': '5.kml', 'punto': 109, 'latitude': 39.46126, 'longitude': -0.34924}])])
((39.46136, -0.34954), [([{'id_oferta': '1.kml', 'punto': 2, 'latitude': 39.46136, 'longitude': -0.34954}, {'id_oferta': '12.kml', 'punto': 137, 'latitude': 39.46136, 'longitude': -0.34954}, {'id_oferta': '5.kml', 'punto': 110, 'latitude': 39.46136, 'longitude': -0.34954}], [{'id_demanda': '1.kml', 'punto': 2, 'latitude': 39.46136, 'longitude': -0.34954}, {'id_demanda': '12.kml', 'punto': 137, 'latitude': 39.46136, 'longitude': -0.34954}, {'id_demanda': '5.kml', 'pu

Con el cogroup by key agrupamos todos los datos de los dos subconjuntos en en una lista de datos que tienen la misma clave.
Tengo que probar tambien:
 combined_pcollection = (ofertas_vehiculos, demandas_vehiculos) | beam.CombinePerKey(lambda x, y: (sum(x), sum(y)))
Y si no nos genera el resultado esperado, intentar crar un Dofn que nos dé el resultado que esperamos.

